<a href="https://colab.research.google.com/github/eridanletalis/ML-Buisness/blob/master/hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>